In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.neighbors import LocalOutlierFactor
from sklearn.ensemble import IsolationForest
from sklearn.svm import OneClassSVM
from sklearn.metrics import roc_curve, auc
from scipy.io import loadmat
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
from sklearn.metrics import precision_score, recall_score, f1_score, roc_curve, auc
import time

In [2]:
data = loadmat('datasets/musk.mat')

X = data['X']
y = data['y'].flatten()

print(f"Kształt X: {X.shape}")
print(f"Kształt y: {y.shape}")
print(f"Liczba normalnych (0): {np.sum(y == 0)}")
print(f"Liczba anomalii (1): {np.sum(y == 1)}")

Kształt X: (3062, 166)
Kształt y: (3062,)
Liczba normalnych (0): 2965
Liczba anomalii (1): 97


In [3]:

X_normal = X[y == 0]
y_normal = y[y == 0] 

X_train_normal, X_test_normal, y_train_normal, y_test_normal = train_test_split(
    X_normal, y_normal, test_size=0.3, random_state=42
)

X_anomalies = X[y == 1]
y_anomalies = y[y == 1]
X_test = np.vstack([X_test_normal, X_anomalies])
y_test = np.hstack([y_test_normal, y_anomalies])

X_train = X_train_normal
y_train = y_train_normal

print(f"Zbiór treningowy: {X_train.shape[0]} próbek (wszystkie normalne)")
print(f"Zbiór testowy: {X_test.shape[0]} próbek ({np.sum(y_test == 0)} normalne, {np.sum(y_test == 1)} anomalie)")

Zbiór treningowy: 2075 próbek (wszystkie normalne)
Zbiór testowy: 987 próbek (890 normalne, 97 anomalie)


In [4]:
# Dopasowanie scalera do danych treningowych
scaler = StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [5]:
def evaluate_model(model, X_train_scaled, X_test_scaled, y_test, verbose=True):
    # Trenowanie modelu i mierzenie czasu
    start_train_time = time.time()
    model.fit(X_train_scaled)
    train_time = time.time() - start_train_time
    
    # Przewidywanie na danych testowych i mierzenie czasu
    start_test_time = time.time()
    test_predictions = model.predict(X_test_scaled)
    test_scores = model.score_samples(X_test_scaled)
    test_time = time.time() - start_test_time
    
    # Obliczanie metryk dla danych testowych
    precision = precision_score(y_test, test_predictions, pos_label=-1)
    recall = recall_score(y_test, test_predictions, pos_label=-1)
    f1 = f1_score(y_test, test_predictions, pos_label=-1)
    fpr, tpr, _ = roc_curve(y_test, -test_scores)  # Negacja, bo niższe score to anomalie
    roc_auc = auc(fpr, tpr)
    
    if verbose:
        print(f"Precision: {precision:.4f}")
        print(f"Recall: {recall:.4f}")
        print(f"F1-score: {f1:.4f}")
        print(f"AUC dla danych testowych: {roc_auc:.4f}")
        print(f"Czas trenowania: {train_time:.4f} s")
        print(f"Czas testowania: {test_time:.4f} s")
    
    # Zwracamy metryki w słowniku
    return {
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'roc_auc': roc_auc,
        'train_time': train_time,
        'test_time': test_time
    }



In [7]:
import numpy as np
print(np.unique(y_test))

[0. 1.]


In [6]:
# Lista wartości n_neighbors do przetestowania
n_neighbors_list = [5, 10, 20, 30]

# Inicjalizacja listy na wyniki
results_list = []

# Pętla przez wartości n_neighbors
for n in n_neighbors_list:
    # Tworzenie modelu LOF
    lof = LocalOutlierFactor(n_neighbors=n, novelty=True)
    
    # Ocena modelu z wyłączonym wypisywaniem
    results = evaluate_model(lof, X_train_scaled, X_test_scaled, y_test, verbose=False)
    
    # Dodanie wartości n_neighbors do wyników
    results['n_neighbors'] = n
    
    # Zapisanie wyników
    results_list.append(results)

# Tworzenie DataFrame z wynikami
df_results = pd.DataFrame(results_list)

# Ustawienie kolejności kolumn
df_results = df_results[['n_neighbors', 'precision', 'recall', 'f1', 'roc_auc', 'train_time', 'test_time']]

# Wyświetlenie tabeli wyników
print("Wyniki dla różnych wartości n_neighbors:")
print(df_results)

ValueError: Target is multiclass but average='binary'. Please choose another average setting, one of [None, 'micro', 'macro', 'weighted'].